In [1]:
import argparse
import os
import pathlib
import shutil
import sys

import numpy as np
import tqdm

cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd
else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break
sys.path.append(str(root_dir / "utils"))
from arg_parsing_utils import check_for_missing_args, parse_args
from file_reading import read_zstack_image
from notebook_init_utils import bandicoot_check, init_notebook

root_dir, in_notebook = init_notebook()

image_base_dir = bandicoot_check(
    pathlib.Path(os.path.expanduser("~/mnt/bandicoot")).resolve(), root_dir
)

sys.path.append(str(pathlib.Path(f"{root_dir}/utils").resolve()))
from file_checking import check_number_of_files

In [2]:
if not in_notebook:
    args = parse_args()
    well_fov = args["well_fov"]
    patient = args["patient"]
    input_subparent_name = args["input_subparent_name"]
    mask_subparent_name = args["mask_subparent_name"]
    check_for_missing_args(
        well_fov=well_fov,
        patient=patient,
        input_subparent_name=input_subparent_name,
        mask_subparent_name=mask_subparent_name,
    )
else:
    patient = "NF0014_T1"
    well_fov = "C4-2"
    input_subparent_name = "convolution_25"
    mask_subparent_name = "convolution_25_segmentation_masks"

In [3]:
# set path to the processed data dir
segmentation_data_dir = pathlib.Path(
    f"{image_base_dir}/data/{patient}/{mask_subparent_name}/{well_fov}"
).resolve(strict=True)
zstack_dir = pathlib.Path(
    f"{image_base_dir}/data/{patient}/{input_subparent_name}/{well_fov}"
).resolve(strict=True)

In [4]:
# perform checks for each directory
segmentation_data_files = list(segmentation_data_dir.glob("*"))

## Copy files from processed dir to cellprofiler images dir

In [5]:
masks_names_to_keep_dict = {
    "cell_masks_watershed.tiff": "cell_masks.tiff",
    "cytoplasm_mask.tiff": "cytoplasm_masks.tiff",
    "nuclei_masks_reassigned.tiff": "nuclei_masks.tiff",
    "organoid_masks_reconstructed.tiff": "organoid_masks.tiff",
}

In [6]:
# remove files that are not in the list of masks to keep
for file in tqdm.tqdm(segmentation_data_files):
    # check if the file is in the masks_names_to_keep_dict as a key or value
    if (
        file.name not in masks_names_to_keep_dict.keys()
        and file.name not in masks_names_to_keep_dict.values()
    ):
        # if not, remove the file
        file.unlink()
        print(f"Removed file: {file.name}")
    else:
        # rename the file to the new name
        if file.name in masks_names_to_keep_dict.keys():
            new_name = masks_names_to_keep_dict[file.name]
            new_file_path = segmentation_data_dir / new_name
            if not new_file_path.exists():
                file.rename(new_file_path)
                print(f"Renamed file: {file.name} to {new_name}")
            else:
                print(f"File {new_name} already exists, skipping rename.")
        else:
            print(f"File {file.name} already exists, skipping rename.")

  0%|          | 0/13 [00:00<?, ?it/s]

File organoid_masks.tiff already exists, skipping rename.


 46%|████▌     | 6/13 [00:00<00:00, 57.06it/s]

Removed file: nuclei_masks_reconstructed_corrected.tiff
Removed file: nuclei_masks_reconstructed.tiff
Removed file: organoid_masks_reconstructed_corrected.tiff
Removed file: organoid_reconstruction_dict.npy
Removed file: cell_masks_corrected.tiff
Removed file: nuclei_masks_decoupled.tiff
File organoid_masks.tiff already exists, skipping rename.
Renamed file: cytoplasm_mask.tiff to cytoplasm_masks.tiff
Removed file: nuclei_reconstruction_dict.npy


 92%|█████████▏| 12/13 [00:00<00:00, 53.75it/s]

Renamed file: nuclei_masks_reassigned.tiff to nuclei_masks.tiff
Renamed file: cell_masks_watershed.tiff to cell_masks.tiff


100%|██████████| 13/13 [00:00<00:00, 55.50it/s]

Removed file: organoid_masks_decoupled.tiff


In [7]:
# regrab the segmentation data files after renaming
segmentation_data_files = list(segmentation_data_dir.glob("*"))

In [8]:
# copy the masks to the zstack directory
for file in tqdm.tqdm(segmentation_data_files):
    for original_name, new_name in masks_names_to_keep_dict.items():
        if file.name == new_name:
            destination = zstack_dir / new_name
            shutil.copyfile(file, destination)
            print(f"Copied file: {file} to {destination}")

 40%|████      | 2/5 [00:03<00:05,  1.76s/it]

Copied file: /home/lippincm/mnt/bandicoot/NF1_organoid_data/data/NF0014_T1/convolution_25_segmentation_masks/C4-2/nuclei_masks.tiff to /home/lippincm/mnt/bandicoot/NF1_organoid_data/data/NF0014_T1/convolution_25/C4-2/nuclei_masks.tiff


 60%|██████    | 3/5 [00:05<00:03,  1.66s/it]

Copied file: /home/lippincm/mnt/bandicoot/NF1_organoid_data/data/NF0014_T1/convolution_25_segmentation_masks/C4-2/organoid_masks.tiff to /home/lippincm/mnt/bandicoot/NF1_organoid_data/data/NF0014_T1/convolution_25/C4-2/organoid_masks.tiff


 80%|████████  | 4/5 [00:06<00:01,  1.64s/it]

Copied file: /home/lippincm/mnt/bandicoot/NF1_organoid_data/data/NF0014_T1/convolution_25_segmentation_masks/C4-2/cell_masks.tiff to /home/lippincm/mnt/bandicoot/NF1_organoid_data/data/NF0014_T1/convolution_25/C4-2/cell_masks.tiff


100%|██████████| 5/5 [00:09<00:00,  1.94s/it]

Copied file: /home/lippincm/mnt/bandicoot/NF1_organoid_data/data/NF0014_T1/convolution_25_segmentation_masks/C4-2/cytoplasm_masks.tiff to /home/lippincm/mnt/bandicoot/NF1_organoid_data/data/NF0014_T1/convolution_25/C4-2/cytoplasm_masks.tiff
